<a href="https://colab.research.google.com/github/ssnirgudkar/Home-AIML/blob/main/IR_1500_Autolabel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This code will be run for auto labeling the 1500 IR images on segments.ai. 
#It will use detectron, we will train the detectron using the Pilot IR, and then each of the batches within the 1500 images. 
#This way each time it will learn from the segmentation done on each batch and improve it

In [ ]:
pip install --upgrade segments-ai

In [ ]:
#Establishing the api key for my segments.ai account 
from segments import SegmentsClient
api_key = "a89182567b17766b91773021b18d04574cd75109"
client = SegmentsClient(api_key)

In [18]:
# We will start with pilot IR 100 images that were handmade segmented as the start. Later on, we will add release file for each batch
dataset_identifier = "ssnirgudkar/PilotIR"


In [ ]:
#install dependencies for detectron 2
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__,torch.cuda.is_available())
!gcc --version



In [ ]:
!pip install torch==1.8.1

In [ ]:
#install detectron2 (Colab has CUDA 10.1 + torch 1.8)
import torch 
assert torch.__version__.startswith("1.8") #need to manually install 1.8 only if colab changes it's default version. as we see above, the default is 1.8 rt now
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime



In [ ]:
#setup logger for detectron2 
import detectron2 
from detectron2.utils.logger import setup_logger 
setup_logger()

In [ ]:

#clone segments.ai repo for their utils code 
!git clone https://github.com/segments-ai/fast-labeling-workflow/
%cd fast-labeling-workflow


In [ ]:
# Initialize a SegmentsDataset from the release file for handmade manually segmented 100 IR images from pilot IR. 
#Later on release file for each batch will be added to this. 
from segments import SegmentsDataset
from segments.utils import export_dataset

release_file = client.get_release(dataset_identifier,'V2.0')
#print(release_file)
releasedataset = SegmentsDataset(release_file, labelset='segmentation', filter_by='labeled')


In [25]:
# Now train the detectron model using the 100 image segmentation dataset(or additional batch release dataset)
from utils import train_model
model = train_model(releasedataset)

100%|██████████| 103/103 [00:02<00:00, 45.94it/s]


Exported to ./export_ssnirgudkar_PilotIR_V2.0.json. Images and labels in segments/ssnirgudkar_PilotIR/V2.0
[06/03 22:53:38 d2.data.datasets.coco]: Loaded 103 images in COCO format from ./export_ssnirgudkar_PilotIR_V2.0.json
Metadata(evaluator_type='coco', image_root='segments/ssnirgudkar_PilotIR/V2.0', json_file='./export_ssnirgudkar_PilotIR_V2.0.json', name='my_dataset', thing_classes=['object', 'sky', 'water', 'warm entity', 'background', 'cold entity', 'sun'])
[06/03 22:53:49 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), strid

model_final_f10217.pkl: 178MB [00:02, 80.1MB/s]                           
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (8, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (8,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (28, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (28,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to

[06/03 22:53:53 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/detectron2/data/detection_utils.py:419: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  torch.stack([torch.from_numpy(np.ascontiguousarray(x)) for x in masks])
/usr/local/lib/python3.7/dist-packages/detectron2/data/detection_utils.py:419: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. T

[06/03 22:54:01 d2.utils.events]:  eta: 0:01:49  iter: 19  total_loss: 3.392  loss_cls: 1.912  loss_box_reg: 0.6124  loss_mask: 0.6914  loss_rpn_cls: 0.2163  loss_rpn_loc: 0.1129  time: 0.3815  data_time: 0.0179  lr: 4.9953e-06  max_mem: 2231M
[06/03 22:54:09 d2.utils.events]:  eta: 0:01:41  iter: 39  total_loss: 3.547  loss_cls: 1.78  loss_box_reg: 0.5969  loss_mask: 0.6875  loss_rpn_cls: 0.2821  loss_rpn_loc: 0.1554  time: 0.3826  data_time: 0.0088  lr: 9.9902e-06  max_mem: 2280M
[06/03 22:54:17 d2.utils.events]:  eta: 0:01:33  iter: 59  total_loss: 3.151  loss_cls: 1.557  loss_box_reg: 0.5995  loss_mask: 0.6774  loss_rpn_cls: 0.2021  loss_rpn_loc: 0.1294  time: 0.3846  data_time: 0.0087  lr: 1.4985e-05  max_mem: 2280M
[06/03 22:54:24 d2.utils.events]:  eta: 0:01:25  iter: 79  total_loss: 2.99  loss_cls: 1.269  loss_box_reg: 0.6339  loss_mask: 0.6644  loss_rpn_cls: 0.1292  loss_rpn_loc: 0.1554  time: 0.3861  data_time: 0.0074  lr: 1.998e-05  max_mem: 2280M
[06/03 22:54:32 d2.utils.ev

In [26]:
# This is the dataset that we have to infer using the model
dataset_identifier_infer = "ssnirgudkar/IR-2020-10-20-15-52-18"
dataset_infer = client.get_dataset(dataset_identifier_infer)

#dataset_identifier_trial = "ssnirgudkar/trial"
#dataset_infer_trial = client.get_dataset(dataset_identifier_trial)



In [ ]:
# Get images that are not in labeled status for auto-labeling 
infer_release_file = client.get_release(dataset_identifier_infer,'V1.0')
infer_releasedataset = SegmentsDataset(infer_release_file, labelset='ground-truth', filter_by=['Unlabeled','Skipped','Prelabeled'])


#trial_release_file = client.get_release(dataset_identifier_trial,'V1.1')
#trial_releasedataset = SegmentsDataset(trial_release_file, labelset='ground-truth', filter_by=['Unlabeled','Skipped','Prelabeled'])
#trial_releasedataset = SegmentsDataset(trial_release_file, labelset='ground-truth', filter_by=['Prelabeled'])

In [28]:
from segments import SegmentsDataset
from segments.utils import export_dataset
from segments.utils import bitmap2file

import cv2 
import numpy as np 
import os 
from utils import visualize 
#this needs 3 channel image. so had to convert grayscale IR to RGB - 3 channels by copying the same pixel value on all 3
#add logic to check if the image is 1 channel, and if so convert it to 3 channel
for sample in infer_releasedataset:
#for sample in trial_releasedataset:
   infer_image = sample['image'] 
      
   #imgcv = cv2.imread(infer_image)
   #assert not isinstance(imgcv,type(None)), 'image not found'
   #print ("file read = {0}".format(imgcv))
   #print(os.getcwd())  # prints current working directory
   #infer_image2 = RemoveAlphaChannel[infer_image, White]
   #4channel CYMK image is not taken by detectron. it needs 3 channels. so converting to RGB. we can add a conditional check
   # to first check if it's 4 channels and then convert only if it is
   infer_image2 = infer_image.convert("RGB")
   segmentation_bitmap, annotations = model(infer_image2)
   # This message gets printed for each image 
   print("segmentation bitmap executed")
    

# Upload the predictions to Segments.ai so that we can check the effectiveness of auto labels
   file = bitmap2file(segmentation_bitmap)
   asset = client.upload_asset(file, 'label.png')    
   attributes = {
        'format_version': '0.1',
        'annotations': annotations,
        'segmentation_bitmap': { 'url': asset['url'] },
   }
   client.add_label(sample['uuid'], 'ground-truth', attributes, label_status='PRELABELED')
   
  

segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation bitmap executed
segmentation b